In [41]:
import pandas as pd
from datetime import timedelta

In [43]:
input_file = 'MTA_Subway_Hourly_Ridership__2020-2024_20250619.csv'
output_file = 'subway_2023_summarized.csv'

In [45]:
chunk_size = 100_000  # Adjust based on available memory
first_chunk = True 

In [47]:
filtered_chunks = []

In [52]:
for chunk in pd.read_csv(input_file, chunksize=chunk_size, parse_dates=['transit_timestamp']):
    # Filter for 2023 and Manhattan
    filtered_chunk = chunk[
        (chunk['transit_timestamp'].dt.year == 2023) &
        (chunk['borough'] == 'Manhattan')
    ]
    filtered_chunks.append(filtered_chunk)

# Combine and save the filtered data
filtered_df = pd.concat(filtered_chunks, ignore_index=True)
filtered_df.to_csv(output_file, index=False)

print(f"Filtered data saved to {output_file}")

C:\Users\admin2\AppData\Local\Temp\ipykernel_2776\2874455961.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(input_file, chunksize=chunk_size, parse_dates=['transit_timestamp']):
C:\Users\admin2\AppData\Local\Temp\ipykernel_2776\2874455961.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(input_file, chunksize=chunk_size, parse_dates=['transit_timestamp']):
C:\Users\admin2\AppData\Local\Temp\ipykernel_2776\2874455961.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in pd.read_csv(input_file, chunksize=chunk_size, parse_dates=['transit_timestamp']):
C:\Users\admin2\AppData\Local\Temp\ipykernel_2776\2874455961.py:1: UserWarning: Could not infer format, so each element will

Filtered data saved to subway_2023_summarized.csv


In [54]:
input_file = 'subway_2023_summarized.csv'
output_file = 'subway_2023_summarized_final.csv'

In [35]:
df.head()

,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,latitude,longitude,Georeference
0,2023-01-23 23:00:00,subway,144,Dyckman St (A),Manhattan,omny,OMNY - Full Fare,6,0,40.865490,-73.92727,POINT (-73.92727 40.86549)
1,2023-01-23 19:00:00,subway,144,Dyckman St (A),Manhattan,metrocard,Metrocard - Seniors & Disability,1,0,40.865490,-73.92727,POINT (-73.92727 40.86549)
2,2023-01-23 19:00:00,subway,14,"23 St (R,W)",Manhattan,omny,OMNY - Full Fare,419,1,40.741302,-73.98934,POINT (-73.98934 40.741302)
3,2023-01-23 13:00:00,subway,13,"28 St (R,W)",Manhattan,omny,OMNY - Full Fare,145,0,40.745495,-73.98869,POINT (-73.98869 40.745495)
4,2023-01-23 05:00:00,subway,303,157 St (1),Manhattan,omny,OMNY - Full Fare,52,4,40.834040,-73.94489,POINT (-73.94489 40.83404)


In [56]:
df.shape

(7871634, 12)

In [58]:
columns_to_drop = ['transit_mode', 'borough','payment_method','fare_class_category','transfers','Georeference']

In [60]:
first_chunk = True

for chunk in pd.read_csv(input_file, chunksize=chunk_size):
    # Drop unwanted columns if they exist
    chunk = chunk.drop(columns=[col for col in columns_to_drop if col in chunk.columns], errors='ignore')
    
    # Write to output CSV
    chunk.to_csv(output_file, mode='w' if first_chunk else 'a', index=False, header=first_chunk)
    first_chunk = False

In [19]:
chunk_size = 100000  

chunks = [df[i:i + chunk_size] for i in range(0, len(df), chunk_size)]
merged_chunks = []

for chunk in chunks:
    merged_chunk = chunk.merge(df1[['LocationID', 'Borough']], left_on='PULocationID', right_on='LocationID', how='left')
    merged_chunk = merged_chunk.rename(columns={'Borough': 'PULocationID_Borough'})
    merged_chunk = merged_chunk.drop(columns=['LocationID'])  # drop extra join column if you want
    merged_chunks.append(merged_chunk)

df_merged = pd.concat(merged_chunks, ignore_index=True)

In [23]:
chunk_size = 100000  

chunks = [df_merged[i:i + chunk_size] for i in range(0, len(df_merged), chunk_size)]
merged_chunks = []

for chunk in chunks:
    merged_chunk = chunk.merge(df1[['LocationID', 'Zone']], left_on='DOLocationID', right_on='LocationID', how='left')
    merged_chunk = merged_chunk.rename(columns={'Zone': 'DOLocationID_Zone'})
    merged_chunk = merged_chunk.drop(columns=['LocationID'])  # drop extra join column if you want
    merged_chunks.append(merged_chunk)

df_merged = pd.concat(merged_chunks, ignore_index=True)

In [39]:
df_merged.head()

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,PULocationID,DOLocationID,PULocationID_Borough,DOLocationID_Borough
0,0,2,01/01/2023 12:32:10 AM,01/01/2023 12:40:36 AM,1.0,161,141,Manhattan,Manhattan
1,1,2,01/01/2023 12:55:08 AM,01/01/2023 01:01:27 AM,1.0,43,237,Manhattan,Manhattan
2,2,2,01/01/2023 12:25:04 AM,01/01/2023 12:37:49 AM,1.0,48,238,Manhattan,Manhattan
4,4,2,01/01/2023 12:10:29 AM,01/01/2023 12:21:19 AM,1.0,107,79,Manhattan,Manhattan
5,5,2,01/01/2023 12:50:34 AM,01/01/2023 01:02:52 AM,1.0,161,137,Manhattan,Manhattan


In [27]:
df_merged = df_merged[((df_merged['PULocationID_Borough'] == 'Manhattan') | (df_merged['DOLocationID_Borough'] == 'Manhattan'))]

In [41]:
chunk_size = 10_000_000

# Split the DataFrame into chunks
chunks = [df[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

# Optional: Process each chunk (you can modify this section as needed)
processed_chunks = []
for chunk in chunks:
    # Example: Keep only first 8 columns
    chunk = chunk.iloc[:, :8]
    
    # (Other processing can go here)
    
    processed_chunks.append(chunk)

# Combine all processed chunks back into one DataFrame
df_combined = pd.concat(processed_chunks, ignore_index=True)

In [43]:
df_combined.head()

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,PULocationID,DOLocationID
0,0,2,01/01/2023 12:32:10 AM,01/01/2023 12:40:36 AM,1.0,161,141
1,1,2,01/01/2023 12:55:08 AM,01/01/2023 01:01:27 AM,1.0,43,237
2,2,2,01/01/2023 12:25:04 AM,01/01/2023 12:37:49 AM,1.0,48,238
3,3,1,01/01/2023 12:03:48 AM,01/01/2023 12:13:25 AM,0.0,138,7
4,4,2,01/01/2023 12:10:29 AM,01/01/2023 12:21:19 AM,1.0,107,79


In [45]:
chunk_size = 100000  

chunks = [df_combined[i:i + chunk_size] for i in range(0, len(df_combined), chunk_size)]
merged_chunks = []

for chunk in chunks:
    merged_chunk = chunk.merge(df1[['LocationID', 'Zone']], left_on='PULocationID', right_on='LocationID', how='left')
    merged_chunk = merged_chunk.rename(columns={'Zone': 'PULocationID_Zone'})
    merged_chunk = merged_chunk.drop(columns=['LocationID'])  # drop extra join column if you want
    merged_chunks.append(merged_chunk)

df_merged = pd.concat(merged_chunks, ignore_index=True)

In [49]:
chunk_size = 100000  

chunks = [df_merged[i:i + chunk_size] for i in range(0, len(df_merged), chunk_size)]
merged_chunks = []

for chunk in chunks:
    merged_chunk = chunk.merge(df1[['LocationID', 'Zone']], left_on='DOLocationID', right_on='LocationID', how='left')
    merged_chunk = merged_chunk.rename(columns={'Zone': 'DOLocationID_Zone'})
    merged_chunk = merged_chunk.drop(columns=['LocationID'])  # drop extra join column if you want
    merged_chunks.append(merged_chunk)

df_merged = pd.concat(merged_chunks, ignore_index=True)

In [51]:
df_merged.head()

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,PULocationID,DOLocationID,PULocationID_Zone,DOLocationID_Zone
0,0,2,01/01/2023 12:32:10 AM,01/01/2023 12:40:36 AM,1.0,161,141,Midtown Center,Lenox Hill West
1,1,2,01/01/2023 12:55:08 AM,01/01/2023 01:01:27 AM,1.0,43,237,Central Park,Upper East Side South
2,2,2,01/01/2023 12:25:04 AM,01/01/2023 12:37:49 AM,1.0,48,238,Clinton East,Upper West Side North
3,3,1,01/01/2023 12:03:48 AM,01/01/2023 12:13:25 AM,0.0,138,7,LaGuardia Airport,Astoria
4,4,2,01/01/2023 12:10:29 AM,01/01/2023 12:21:19 AM,1.0,107,79,Gramercy,East Village


In [65]:
chunk_size = 50000

chunks = [df_merged[i:i + chunk_size] for i in range(0, len(df_merged), chunk_size)]

processed_chunks = []
for chunk in chunks:
    chunk = chunk.drop(['Unnamed: 0','VendorID'],axis=1)
    processed_chunks.append(chunk)

df_combined = pd.concat(processed_chunks, ignore_index=True)

In [67]:
df_combined.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,PULocationID,DOLocationID,PULocationID_Zone,DOLocationID_Zone
0,01/01/2023 12:32:10 AM,01/01/2023 12:40:36 AM,1.0,161,141,Midtown Center,Lenox Hill West
1,01/01/2023 12:55:08 AM,01/01/2023 01:01:27 AM,1.0,43,237,Central Park,Upper East Side South
2,01/01/2023 12:25:04 AM,01/01/2023 12:37:49 AM,1.0,48,238,Clinton East,Upper West Side North
3,01/01/2023 12:03:48 AM,01/01/2023 12:13:25 AM,0.0,138,7,LaGuardia Airport,Astoria
4,01/01/2023 12:10:29 AM,01/01/2023 12:21:19 AM,1.0,107,79,Gramercy,East Village


In [69]:
df_combined.to_csv('2023_Yellow_Taxi_Trip_Data_20250523_Manhattan.csv')

In [38]:
chunk_size = 50000

chunks = [df[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

processed_chunks = []
for chunk in chunks:
    chunk = chunk.drop(['Unnamed: 0','tpep_pickup_datetime','tpep_dropoff_datetime','tpep_pickup_timestamp','tpep_dropoffup_timestamp'],axis=1)
    processed_chunks.append(chunk)

df = pd.concat(processed_chunks, ignore_index=True)

In [42]:
df['pickup_datetime'] = pd.to_datetime(df['pickupdate'].astype(str) + ' ' + df['pickup_time'].astype(str))
df['dropoff_datetime'] = pd.to_datetime(df['dropoffdate'].astype(str) + ' ' + df['dropoff_time'].astype(str))

# Create two separate DataFrames for pickup and dropoff with common column names
pickup_df = df[['pickup_datetime', 'PULocationID_Zone', 'passenger_count']].rename(
    columns={'pickup_datetime': 'datetime', 'PULocationID_Zone': 'zone'}
)

dropoff_df = df[['dropoff_datetime', 'DOLocationID_Zone', 'passenger_count']].rename(
    columns={'dropoff_datetime': 'datetime', 'DOLocationID_Zone': 'zone'}
)

# Combine both into one DataFrame
combined_df = pd.concat([pickup_df, dropoff_df])



# Group by zone and datetime and sum passenger count
summary = combined_df.groupby(['zone', 'datetime'])['passenger_count'].sum().reset_index()



# Optional: sort by datetime
summary = summary.sort_values(by='datetime')


print(summary)

                             zone            datetime  passenger_count
1350745  Washington Heights South 1970-01-20 10:00:00              1.0
205585               Central Park 2001-01-01 00:00:00              1.0
1323094     Upper West Side South 2001-01-01 00:00:00              2.0
856896             Midtown Center 2001-01-01 00:00:00              2.0
648260                JFK Airport 2001-01-01 00:00:00              2.0
...                           ...                 ...              ...
701563        Kingsbridge Heights 2024-01-03 11:00:00              1.0
983534             Outside of NYC 2024-01-03 17:00:00              4.0
513340               Forest Hills 2024-01-03 19:00:00              4.0
657027                JFK Airport 2024-01-03 20:00:00              1.0
897504                    Midwood 2024-01-03 20:00:00              1.0

[1468064 rows x 3 columns]


In [50]:
summary = summary.reset_index(drop=True)

In [52]:
print(summary)

                             zone            datetime  passenger_count
0        Washington Heights South 1970-01-20 10:00:00              1.0
1                    Central Park 2001-01-01 00:00:00              1.0
2           Upper West Side South 2001-01-01 00:00:00              2.0
3                  Midtown Center 2001-01-01 00:00:00              2.0
4                     JFK Airport 2001-01-01 00:00:00              2.0
...                           ...                 ...              ...
1468059       Kingsbridge Heights 2024-01-03 11:00:00              1.0
1468060            Outside of NYC 2024-01-03 17:00:00              4.0
1468061              Forest Hills 2024-01-03 19:00:00              4.0
1468062               JFK Airport 2024-01-03 20:00:00              1.0
1468063                   Midwood 2024-01-03 20:00:00              1.0

[1468064 rows x 3 columns]


In [54]:
summary.to_csv('yellowcab_summarized.csv')


In [46]:
combined_df.head()

,datetime,zone,passenger_count
0,2023-01-01 01:00:00,Midtown Center,1.0
1,2023-01-01 01:00:00,Central Park,1.0
2,2023-01-01 00:00:00,Clinton East,1.0
3,2023-01-01 00:00:00,LaGuardia Airport,0.0
4,2023-01-01 00:00:00,Gramercy,1.0


In [48]:
combined_df.dtypes

datetime           datetime64[ns]
zone                       object
passenger_count           float64
dtype: object

In [3]:
df1 = pd.read_csv('yellowcab_summarized.csv')

In [7]:
df2 = pd.read_csv('taxi_zone_lookup.csv')

In [9]:
df2.head()

,LocationID,Zone,service_zone,Borough
0,1,Newark Airport,EWR,EWR
1,2,Jamaica Bay,Boro Zone,Queens
2,3,Allerton/Pelham Gardens,Boro Zone,Bronx
3,4,Alphabet City,Yellow Zone,Manhattan
4,5,Arden Heights,Boro Zone,Staten Island


In [11]:
chunk_size = 100000  

chunks = [df1[i:i + chunk_size] for i in range(0, len(df1), chunk_size)]
merged_chunks = []

for chunk in chunks:
    merged_chunk = chunk.merge(df2[['Zone','Borough']], left_on='zone', right_on='Zone', how='left')
    merged_chunk = merged_chunk.rename(columns={'Borough': 'Borough'})
    merged_chunks.append(merged_chunk)

df_merged = pd.concat(merged_chunks, ignore_index=True)

In [13]:
df_merged.head()

,Unnamed: 0,zone,datetime,passenger_count,Zone,Borough
0,0,Washington Heights South,1970-01-20 10:00:00,1.0,Washington Heights South,Manhattan
1,1,Central Park,2001-01-01 00:00:00,1.0,Central Park,Manhattan
2,2,Upper West Side South,2001-01-01 00:00:00,2.0,Upper West Side South,Manhattan
3,3,Midtown Center,2001-01-01 00:00:00,2.0,Midtown Center,Manhattan
4,4,JFK Airport,2001-01-01 00:00:00,2.0,JFK Airport,Queens


In [15]:
df_merged = df_merged[(df_merged['Borough'] == 'Manhattan')]

In [17]:
df_merged.head()

,Unnamed: 0,zone,datetime,passenger_count,Zone,Borough
0,0,Washington Heights South,1970-01-20 10:00:00,1.0,Washington Heights South,Manhattan
1,1,Central Park,2001-01-01 00:00:00,1.0,Central Park,Manhattan
2,2,Upper West Side South,2001-01-01 00:00:00,2.0,Upper West Side South,Manhattan
3,3,Midtown Center,2001-01-01 00:00:00,2.0,Midtown Center,Manhattan
6,6,East Chelsea,2001-01-01 14:00:00,1.0,East Chelsea,Manhattan


In [19]:
df_merged.to_csv('yellowCab_summarized_manhattan_data.csv')

In [21]:
df_merged.shape

(542740, 6)